# Set Up

In [ ]:
! pip install -qqq bitsandbytes

In [ ]:
import sys
from google.colab import drive
import os

drive.mount('/content/drive')
drive_path = os.path.join('drive', 'MyDrive', 'Colab Notebooks', 'Dacon', 'SentenceOrder') # 다른 드라이브에서 사용할 경우, 이 부분만 적절히 수정

if drive_path not in sys.path:
  sys.path.append(drive_path)
from utility import *

CONFIG_PATH = drive_path + '/config.yaml'
config = load_config(CONFIG_PATH)

In [ ]:
train = pd.read_csv(drive_path + '/dataset/train.csv').drop(columns = 'ID')

# Load the Tokenizer and the Model

In [ ]:
bnb_config = BitsAndBytesConfig(**config['bnb_config'])

model = AutoModelForCausalLM.from_pretrained(
  config['augment_model_name'], # Qwen/Qwen3-14B
  quantization_config=bnb_config,
  **config['auto_model']
)

model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(config['augment_model_name'], use_fast=True)

# Augmentation

오리지널 데이터처럼, 무작위로 섞인 네 개의 문장을 정답과 함께 생성하는 것은 어려운 작업이다. 하지만 단순히 네 개의 문장으로 이루어진 한국어 텍스트를 생성하는 건 LLM에게 매운 쉬운 작업이다.

따라서, 데이터를 증강할 때 오리지널 데이터에 있는 문장을 정렬한 후, LLM에게 "이런 식으로 네 개의 한국어 문장으로 이루어진 글을 생성해 줘" 라고 부탁한다. 이렇게 만들어진 네 개의 문장을 섞는 건 쉬운 일이므로 증강 이후 진행한다. 이런 방식을 취하면 많은 수의 데이터를 간단한 프롬프트 엔지니어링으로 생성할 수 있다.

In [ ]:
def generate_single_data(row, chat_template, tokenizer, model, config):
  # 하나의 데이터를 생성하는 함수

  input = tokenizer(
      [ chat_template.format(row) ],
      return_tensors="pt"
  ).to("cuda")

  output = model.generate(
      input_ids=input.input_ids,
      attention_mask=input.attention_mask,
      eos_token_id=tokenizer.eos_token_id,
      **config['augment_config']
  )

  generated_ids = output[0][len(input.input_ids[0]):]
  response = tokenizer.decode(generated_ids, skip_special_tokens=True)
  return response

## First augmentation



In [ ]:
set_all_seed(42) # 시드 설정

In [ ]:
# 첫 번째 증강에서는 영어 프롬프트 / 너무 강한 제약은 X

prompt = """Your Persona: You are the "Korean Micro-Paragraph Architect." You are exceptionally skilled at understanding patterns from an example I provide and then generating new, original, high-quality Korean texts that mirror its style and structure, while exploring a closely related theme.

Your Mission: I am going to provide you with **an example** of the kind of four-sentence Korean paragraph I am looking for. Please carefully study **this example** to thoroughly understand its structure, tone, coherence, logical flow, the typical level of detail, and **importantly, the specific theme and topic it covers.**

---
Here are the example:

Example: {}
---

Now, having thoroughly analyzed the example I've provided above, your primary task is to generate a NEW and UNIQUE self-contained passage. This passage must consist of exactly four (4) Korean sentences.

Your generated passage should emulate the quality, style, and logical structure demonstrated in the provided example. For the subject matter, **your task is to thoughtfully consider the specific theme and topic of the single example I've provided. Based on this one example, please formulate a topic for your new paragraph that is similar or closely related to it.**

Key Requirements for Your Generated Output:

1. The entire text must be in **Korean.**
2. The text must consist of **precisely four sentences.**
3. The sentences must be thematically connected and exhibit a clear logical progression (e.g., introduction -> supporting detail / elaboration -> further detail / example -> conclusion or summary), as exemplified in the text I showed you.
4. The topic for your generated paragraph will be **similar or closely related to the topic of the single provided example.** However, this chosen topic must be developed with **significant creativity and originality in its content.** The four sentences you craft should present a fresh perspective or new information on this related theme, ensuring your paragraph is entirely distinct from the literal text of the provided example and adds new value. It should not be a mere rephrasing or direct copy of the provided example.

Please proceed to craft your new and unique four-sentence Korean micro-paragraph, with a topic similar or closely related to the provided example.
"""

In [ ]:
message = [
    {"role": "system", "content": prompt}
]

inference_chat_template = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True, enable_thinking=False)

In [ ]:
first_augmented_data = []
num_of_augmented_data = 2131 # 첫 번째 증강 데이터 수

for i in tqdm(range(num_of_augmented_data) ):
  row = train.iloc[random.sample(range(len(train)), 1)]
  sentence = convert_row_to_sentence(row.iloc[0]) # convert_row_to_sentence 함수를 통해서 섞인 네 개의 문장을 정렬하여 하나의 텍스트로 변환
  response = generate_single_data(sentence, inference_chat_template, tokenizer, model, config) # 데이터 생성
  first_augmented_data.append(response)

  if i % 10 == 0: # 많은 시간이 걸리므로, 데이터를 10개 생성할 때마다 저장
    with open(drive_path + '/dataset/first_augmented_data.pkl', 'wb') as f:
      pickle.dump(first_augmented_data, f)

## Second augmentation

In [ ]:
set_all_seed(43) # 시드 설정, 첫 번째 증강과는 다르게

In [ ]:
# 두 번째 증강에서는 한국어 프롬프트 / 부사와 접속사 등을 최대한 쓰지 않도록 제약을 두어 더 어려운 데이터를 생성

prompt = """# 역할
당신은 특정 주제에 대해 독창적인 정보 글을 작성하는 전문 저술가입니다.

# 임무
아래 '[입력 텍스트]'를 **오직 '글의 주제'를 파악하는 용도로만 참고**하세요. 그 후, 파악된 주제에 대해 **당신이 가진 지식을 바탕으로 완전히 새로운 네(4) 개의 문장**을 창조하세요.

# 핵심 지침
1.  **원문 내용 무시**: 입력 텍스트의 구체적인 내용, 구조, 단어는 **의도적으로 무시하고 사용하지 마세요.** 오직 '이 글이 무엇에 대한 글인가?'라는 주제만 가져와야 합니다.
2.  **새로운 정보 생성**: 파악된 주제에 대해, 원문에는 없던 새로운 측면의 사실들을 제시하세요. (예: 역사적 배경, 과학적 원리, 사회적 영향, 미래 전망 등)
3.  **객관적 문체 유지**: 문장은 철저하게 백과사전처럼 객관적이고 정보 중심적으로 작성해야 합니다. 감성, 비유, 개인적 의견은 절대 포함하지 마세요.

# 제약 조건
1.  **네 개의 문장**: 결과물은 반드시 네(4) 개의 한국어 문장으로 구성되어야 합니다.
2.  **문장 연결어 사용 금지**: 각 문장이 완전히 독립적으로 느껴지도록, **'또한', '게다가'** 같은 부사를 포함하여 '그리고', '그러나' 등 문장과 문장을 의미적으로 연결하는 모든 표현의 사용을 금지하세요. **한 문장을 읽고 다음 문장을 예측할 수 없도록 문장 간의 연결고리를 최대한 끊어내야 합니다.**

# 입력 텍스트
{}

# 결과물 /no_think
"""

In [ ]:
message = [
    {"role": "system", "content": prompt}
]

inference_chat_template = tokenizer.apply_chat_template(message, tokenize=False, add_generation_prompt=True, enable_thinking=False)

In [ ]:
second_augmented_data = []
num_of_augmented_data = 2000 # 두 번째 증강 데이터 수

for i in tqdm(range(num_of_augmented_data) ):
  row = train.iloc[random.sample(range(len(train)), 1)]
  sentence = convert_row_to_sentence(row.iloc[0]) # convert_row_to_sentence 함수를 통해서 섞인 네 개의 문장을 정렬하여 하나의 텍스트로 변환
  response = generate_single_data(sentence, inference_chat_template, tokenizer, model, config) # 데이터 생성
  second_augmented_data.append(response)

  if i % 10 == 0: # 많은 시간이 걸리므로, 데이터를 10개 생성할 때마다 저장
    with open(drive_path + '/dataset/second_augmented_data.pkl', 'wb') as f:
      pickle.dump(second_augmented_data, f)